In [24]:
import requests
import random
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import ast
import json
import time
import datetime
from gdeltdoc import GdeltDoc, Filters
import re
import datetime

In [25]:
# pip install gdeltdoc

In [26]:
def fmt(p,n=4):
    return '{0:.{1}f}'.format(p, n)

### GDELT Test

In [32]:
newsOutlet = 'pbsnews.com'
start_date, end_date = ("2022-01-01", "2022-03-15")
f = Filters(
    keyword = "ukraine",
    start_date = start_date,
    end_date = end_date,
    domain_exact = newsOutlet,
    num_records = 250
)

gd = GdeltDoc()

articles = gd.article_search(f)
articles.head()

""


In [18]:
# url = random.choice(articles['url'])
url = 'https://apnews.com/article/russia-ukraine-putin-business-europe-black-sea-224009801161d899abb0cc94c9b17d2f'
soup = BeautifulSoup(requests.get(url).content, "html.parser")
print(url)
print(soup)

https://apnews.com/article/russia-ukraine-putin-business-europe-black-sea-224009801161d899abb0cc94c9b17d2f
<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link data-rh="true" href="/manifest.json" rel="manifest"/><link data-rh="true" href="/branding/favicon/16.png" rel="icon" sizes="16x16"/><link data-rh="true" href="/branding/favicon/24.png" rel="icon" sizes="24x24"/><link data-rh="true" href="/branding/favicon/32.png" rel="icon" sizes="32x32"/><link data-rh="true" href="/branding/favicon/64.png" rel="icon" sizes="64x64"/><link data-rh="true" href="/branding/favicon/128.png" rel="icon" sizes="128x128"/><link data-rh="true" href="/branding/favicon/256.png" rel="icon" sizes="256x256"/><link data-rh="true" href="/branding/apple-touch/ap-news-apple-touch-152.png" rel="apple-touch-icon"/><link data-rh="true" href="/branding/apple-touch/ap-news-apple-touch-152.png" rel="apple-touch-icon" sizes="152x152"/><lin

## NYTimes API

In [ ]:
# pip install --upgrade pynytimes

In [ ]:
# !pip3 install requests-html

In [ ]:
from pynytimes import NYTAPI
nyt = NYTAPI("rrHIfoqwqciGjJ4C5OGeE9p79VdvzOfp", parse_dates=True)

In [ ]:
import datetime

articles = nyt.article_search(
    query = "Ukraine",
    results = 30,
    dates = {
        "begin": datetime.datetime(2022, 2, 24),
        "end": datetime.datetime(2022, 3, 24)
    },
    options = {
        "sort": "oldest",
        "sources": [
            "New York Times",
            # "AP",
            # "Reuters",
            # "International Herald Tribune"
        ],
        # "news_desk": [
        #     "Politics"
        # ],
        # "type_of_material": [
        #     "News Analysis"
        # ]
    }
)

# Article scrape

## Handle scraped text

In [29]:
def get_article_text(soup, newsOutlet, url):
    # Prensa Latina (Cuba)
    if newsOutlet == 'plenglish.com':
        elts = soup.find_all('div', attrs={'class' : 'elementor-widget-container'})
        first_sent = ''
        for e in elts:
            if '(Prensa Latina)' in e.text:
                first_sent = e.text.strip()
        ps = soup.find_all('p')
        p_text = [first_sent]
        for p in ps:
            if p.text.count('/') >= 2:
                break
            if 'class' in p.attrs:
                break
            p_text.append(p.text)
            
    # Al Jazeera (Qatar)
    if newsOutlet == 'aljazeera.com':
        ps = soup.find_all('p')
        p_text = []
        if len(ps) > 4:
            for p in ps:
                if 'Follow Al Jazeera English' not in p.text:
                    p_text.append(p.text.strip())
        else:
            uls = soup.find_all('ul')
            for ul in uls:
                lis = ul.find_all('li')
                for li in lis:
                    if '.' in li.text:
                        p_text.append(li.text.strip())
    # People's Daily (China)
    if newsOutlet == 'en.people.cn':
        ps = soup.find_all('p')
        p_text = [p.text.strip().replace("\'", "'") for p in ps]
    # BBC (UK)
    if newsOutlet == 'bbc.co.uk':
        p_text = []
        lastfour = False
        ps = soup.find_all('p')
        endat = len(ps)
        for i, p in enumerate(ps):
            if 'class' in p.attrs and len(p.attrs['class']) > 0 and 'Paragraph' in p.attrs['class'][0]:
                if 'You may also be interested in' in p.text:
                    endat = i
                    break
                else:
                    lastfour = True
                    
        if lastfour:
            for p in ps[:-6]:
                if 'class' in p.attrs and len(p.attrs['class']) > 0 and 'Paragraph' in p.attrs['class'][0]:
                    p_text.append(p.text.strip())
        else:
            for p in ps[:i]:
                if 'class' in p.attrs and len(p.attrs['class']) > 0 and 'Paragraph' in p.attrs['class'][0]:
                    p_text.append(p.text.strip())
                    
        if 'sport' in url:
            p_text = []
            intros = soup.find_all(attrs={'class', 'qa-introduction'})
            ps = soup.find_all('p')
            for p in ps:
                if 'Last updated on' not in p.text:
                    if 'class' in p.attrs and p.attrs['class'] == []:
                        p_text.append(p.text)
                    elif 'class' in p.attrs and 'qa-introduction' in p.attrs['class']:
                        p_text.append(p.text)
                    else:
                        continue
        if 'newsround' in url:
            ps = soup.find_all('p', attrs={'class': 'newsround-story-body__text'})
            p_text = [p.text for p in ps]
        try:
            removeIndex = p_text.index('This video can not be played')
            p_text.pop(removeIndex)
        except:
            removeIndex = None

    # The Punch (Nigeria)
    if newsOutlet == 'punchng.com': 
        try:
            postContent = soup.find_all(attrs={'class','post-content'})[0]
            ad = "(adsbygoogle = window.adsbygoogle || []).push({});"
            postText = postContent.text.replace(ad, '').strip()
        except:
            postText = ''
        # take out related News
        x = re.search(r"Related News.*?\n", postText) 
        if x:
            relatedNews = x.group(0)
            postText = postText.replace(relatedNews.strip(), '')
        copyrightPunch1 = "Copyright PUNCH"
        copyrightPunch2 = "All rights reserved. This material, and other digital content on this website, may not be reproduced, published, broadcast, rewritten or redistributed in whole or in part without prior express written permission from PUNCH."
        postText = postText.replace(copyrightPunch1,'')
        postText = postText.replace(copyrightPunch2,'')
        p_text = [postText]

    # CNN (US)
    if newsOutlet == 'cnn.com':
        # [print(p.text) for p in soup.find_all(attrs={'class':'zn-body__paragraph'})]

        ps = soup.find_all(attrs={'class' : 'zn-body__paragraph'})
        if ps == []:
            ps = soup.find_all(attrs={'class' : 'BasicArticle__paragraph'})
            
        p_text = [p.text.strip() for p in ps if p.text != '(CNN)']
        
    # Breitbart (US)
    if newsOutlet == 'breitbart.com':
        ps = soup.find_all('p')
        p_text = [p.text.replace(u'\xa0', u' ') for p in ps[:-5] if 'on Facebook and Twitter.' not in p.text]
        
    # Sputnik (Russia)
    if newsOutlet == 'sputniknews.com':
        try:
            articleAnnounceText = soup.find_all('div', {'class' : 'article__announce-text'})[0].text
            if articleAnnounceText == None:
                articleAnnounceText = ''
        except:
            articleAnnounceText = ''
            
        article_blocks = soup.find_all("div", {"class": "article__block"})
        p_text = [articleAnnounceText]
        for ab in article_blocks:
            if "Sputnik's live feed and Telegram channels" in ab.text:
                break
            if '@context' in ab.text:
                continue
            p_text.append(ab.text.strip())
    # Pravda (Russia)
    if newsOutlet == 'english.pravda.ru':
        ps = soup.find_all('p')
        p_text = []
        for p in ps:
            if 'Subscribe to Pravda.Ru Telegram channel, Facebook, Twitter, YouTube, RSS!' in p.text:
                break
            p_text.append(p.text)
        
    # Fox News (US)
    if newsOutlet == 'foxnews.com':
        try:
            json_object = json.loads(soup.find_all('script')[3].text, strict=False)
            jsontext = json_object['articleBody'].replace('&nbsp;  ','\n\n').replace('&nbsp;',' ')
            # Take out ALLCAPS
            matches = re.findall(r'([A-Z][^a-z]+ )+', jsontext)
            toremove = [match for match in matches if len(match.strip().split(' ')) > 1]
            for x in toremove:
                jsontext = jsontext.replace(x, '')
            p_text = [jsontext]
        except:
            print(url)
            print('\t',soup.find_all('script'))
            p_text = ['']
    # RT (Russia)
    if newsOutlet == 'rt.com':
        try:
            article_text_div = soup.find_all('div', attrs={'class': 'article__text'})[0]
            ps = article_text_div.find_all('p')
            p_text = [p.text.replace(u'\xa0', u' ') for p in ps if p.text != 'The statements, views and opinions expressed in this column are solely those of the author and do not necessarily represent those of RT.']
        except:
            p_text = []
        
    # TASS (Russia)
    if newsOutlet == 'tass.com':
        text_block = soup.find(attrs={'class' : 'text-block'})
        ps = text_block.find_all('p')
        p_text = [p.text for p in ps]
        
    # moscowtimes
    if newsOutlet == 'themoscowtimes.com':
        try:
            article_content = soup.find('div', attrs={'class':'article__content'})
            ps = article_content.find_all('p')
            p_text = [p.text for p in ps]
        except:
            p_text = []
            
    if newsOutlet == 'en.interfax.com.ua':
        try:
            article_content = soup.find('div', attrs={'class','article-content'})
            ps = article_content.find_all('p')
            p_text = [p.text for p in ps]
        except:
            p_text = []
    if newsOutlet == 'news.yahoo.com':
        try:
            div = soup.find(attrs={'class':'caas-body'})
            ps = div.find_all('p')
            p_text = [p.text for p in ps]
        except:
            print('problem on',url)
    if newsOutlet == 'cbsnews.com':
        content_body = soup.find('section')
        p_text = [p.text for p in content_body.find_all('p')]
        
    if newsOutlet == 'reuters.com':
        ps = soup.find_all('p')
        p_text = []
        for p in ps:
            if 'Writing by' in p.text:
                break
            p_text.append(p.text)
            
    if newsOutlet == 'apnews.com':
        try:
            article = soup.find(attrs={'class':'Article'})
            ps = article.find_all('p')
            p_text = [p.text for p in ps]
        except:
            try:
                ps = soup.find_all('p')
                p_text = [p.text for p in ps]
            except:
                print(url)
        
    try:
        text = ' '.join([p.strip() for p in p_text])
    except:
        text = ''
        
    if not isinstance(text,str):
        text = ''
    return text

## Web Scrape Function

In [30]:
# newsOutletList = ['plenglish.com','aljazeera.com','en.people.cn','bbc.co.uk',
#                   'punchng.com','cnn.com','breitbart.com','sputniknews.com','english.pravda.ru','foxnews.com',
#                  'riotimesonline.com','rt.com','tass.com']
newsOutletList = ['plenglish.com','aljazeera.com','en.people.cn','bbc.co.uk',
                  'punchng.com','cnn.com','breitbart.com','sputniknews.com','english.pravda.ru',
                  'foxnews.com','rt.com','tass.com','themoscowtimes.com','kyivpost.com','kyivindependent.com']


russianNews = ['themoscowtimes.com','sputniknews.com','rt.com','tass.com','english.pravda.ru']
ukrainianNews = ['en.interfax.com.ua']
northatlanticNews = ['foxnews.com','cnn.com','bbc.co.uk']
otherNews = ['punchng.com','plenglish.com','en.people.cn']


rangeList = [
    ("2022-01-01", "2022-01-31"),
    ("2022-02-01", "2022-02-29"),
    ("2022-03-01", "2022-03-31"),
    ("2022-04-01", "2022-04-30"),
    ("2022-05-01", "2022-05-31"),
    ("2022-06-01", "2022-06-30"),
]
extendedRangeList = [
    ("2022-01-01", "2022-01-15"),    
    ("2022-01-16", "2022-01-31"),
    ("2022-02-01", "2022-02-15"),
    ("2022-02-16", "2022-02-29"),
    ("2022-03-01", "2022-03-15"),
    ("2022-03-16", "2022-03-31"),
    ("2022-04-01", "2022-04-15"),
    ("2022-04-16", "2022-04-30"),
    ("2022-05-01", "2022-05-15"),
    ("2022-05-16", "2022-05-31"),
    ("2022-06-01", "2022-06-15"),
    ("2022-06-16", "2022-06-30"),
]
shortList = [
    ("2022-01-01", "2022-06-30")
]
preWarRangeListExtended = [
    ("2021-10-01", "2021-10-15"), 
    ("2021-10-16", "2021-10-31"), 
    ("2021-11-01", "2021-11-15"), 
    ("2021-11-16", "2021-11-30"), 
    ("2021-12-01", "2021-12-15"), 
    ("2021-12-16", "2021-12-31"),    
    ("2022-01-01", "2022-01-15"),    
    ("2022-01-16", "2022-01-31"),
    ("2022-02-01", "2022-02-15"),
    ("2022-02-16", "2022-02-29"),
    ("2022-03-01", "2022-03-15"),
]
# preWarRangeList = [
#     ("2021-10-01", "2022-10-31"), 
#     ("2021-11-01", "2022-11-30"), 
#     ("2021-12-01", "2022-12-31"),    
#     ("2022-01-01", "2022-01-31"),
#     ("2022-02-01","2022-02-29"),
#     ("2022-03-01", "2022-03-31"),
# ]

In [31]:
newsDict = dict()

In [22]:
chooseNewsOutlets = ['apnews.com']
timeRange = preWarRangeListExtended

writeAllOutletList = False
startStart = time.time()
unscrapedLinks = dict()
for newsOutlet in chooseNewsOutlets:
    startOutlet = time.time()
    newsDict.setdefault(newsOutlet, [])
    unscrapedLinks.setdefault(newsOutlet, [])
    print(newsOutlet)
    totalparsed = 0
    for start_date, end_date in timeRange:
        parsed = 0
        f = Filters(
            keyword = "ukraine",
            start_date = start_date,
            end_date = end_date,
            domain_exact = newsOutlet,
            num_records = 250
        )

        gd = GdeltDoc()

        # Search for articles matching the filters
        articles = gd.article_search(f)
        articleDictList = articles.to_dict('records')

        start = time.time()
        for i, artRec in enumerate(articleDictList):
            if i % 50 == 0:
                current_time = time.time()
                print(f'\t{i}/{len(articleDictList)} articles in {newsOutlet} parsed from {start_date} to {end_date} in {fmt((current_time - start)/60,2)} min.')

            url = artRec['url']

            # skip over unscrapable articles
            skipArticle = False
            badCombos = [('cnn.com', 'live-news'),('aljazeera.com', 'live-news'),
                        ('sputniknews.com','live-updates'), ('bbc.co.uk','programmes'), ('bbc.co.uk','news/live'),
                         ('foxnews.com', 'live-news'),('cbsnews.com','/team/'),('cbsnews.com','/pictures/')]
            for badOutlet, badString in badCombos:
                if newsOutlet == badOutlet and badString in url:
                    skipArticle = True
            if skipArticle:
                unscrapedLinks[newsOutlet].append(artRec)
                continue

            # get article text
            problemOnSoup = False
            try:
                soup = BeautifulSoup(requests.get(url).content, "html.parser")
            except:
                problemOnSoup = True
            if problemOnSoup:
                print(f'\t\tProblem on {url}')
                continue
                
            text = get_article_text(soup,newsOutlet,url)
    
            # get date seen
            date = artRec['seendate']
            yr = int(date[:4])
            month = int(date[4:6])
            day = int(date[6:8])
            d = datetime.date(yr, month, day)
            utc = d.strftime("%s")  # <-- THIS IS THE CODE YOU WANT

            newsDict[newsOutlet].append(
            {'newsOutlet' : newsOutlet,
             'dateSeen' : utc,
             'url' : url,
             'title' : artRec['title'],
             'language' : artRec['language'],
             'sourceCountry' : artRec['sourcecountry'],
             'text' : text
            })
            parsed += 1
            totalparsed += 1
    endOutlet = time.time()
    print(f'\t{totalparsed} articles in {newsOutlet} parsed in {fmt((endOutlet-startOutlet)/60,2)} min.')
    
    # Write dataframes to Excel
    todaysdate = datetime.datetime.fromtimestamp(time.time()).strftime('%m-%d-%Y')
    df = pd.DataFrame.from_records(newsDict[newsOutlet])
    df.to_excel(f"preWarData/{newsOutlet}_{todaysdate}_trial.xlsx")
    print(f'\t{newsOutlet} written to Excel.')
    
if writeAllOutletList:
    allRecords = []
    for newsO in sorted(newsOutletList):
        allRecords = allRecords + newsDict[newsO]
    alldf =  pd.DataFrame.from_records(allRecords)
    alldf.to_excel(f"allOutletData/preWarData/{newsOutlet}_{todaysdate}.xlsx")
    print(f'All dataframes written to Excel.')

endEnd = time.time()
print(f'\nAll outlets parsed in {fmt((endEnd-startStart)/60, 2)} min.')

apnews.com
	0/66 articles in apnews.com parsed from 2021-10-01 to 2021-10-15 in 0.00 min.
	50/66 articles in apnews.com parsed from 2021-10-01 to 2021-10-15 in 0.62 min.
	0/66 articles in apnews.com parsed from 2021-10-16 to 2021-10-31 in 0.00 min.
	50/66 articles in apnews.com parsed from 2021-10-16 to 2021-10-31 in 0.65 min.
	0/61 articles in apnews.com parsed from 2021-11-01 to 2021-11-15 in 0.00 min.
	50/61 articles in apnews.com parsed from 2021-11-01 to 2021-11-15 in 0.56 min.
	0/68 articles in apnews.com parsed from 2021-11-16 to 2021-11-30 in 0.00 min.
	50/68 articles in apnews.com parsed from 2021-11-16 to 2021-11-30 in 0.56 min.
	0/123 articles in apnews.com parsed from 2021-12-01 to 2021-12-15 in 0.00 min.
	50/123 articles in apnews.com parsed from 2021-12-01 to 2021-12-15 in 0.64 min.
	100/123 articles in apnews.com parsed from 2021-12-01 to 2021-12-15 in 1.23 min.
	0/97 articles in apnews.com parsed from 2021-12-16 to 2021-12-31 in 0.00 min.
	50/97 articles in apnews.com p

In [23]:
for outlet in newsDict:
    print(outlet, len(newsDict[outlet]))

apnews.com 1446


In [16]:
random.choice(newsDict[outlet])

{'newsOutlet': 'apnews.com',
 'dateSeen': '1645084800',
 'url': 'https://apnews.com/article/saudi-arabia-keeps-lid-on-oil-output-as-us-gas-prices-rise-e14e8c00d188a4110f73358402e1e595',
 'title': 'Saudi Arabia keeps lid on oil output as US gas prices rise',
 'language': 'English',
 'sourceCountry': 'United States',
 'text': 'DUBAI, United Arab Emirates (AP) — Saudi Arabia is signaling it isn’t willing to pump more oil and won’t push for changes to an agreement with Russia and other producers that has kept a lid on oil production levels. This has Washington concerned as gasoline prices rise and tensions with Russia over Ukraine fuel market uncertainty. The Biden administration dispatched Brett McGurk, the National Security Council’s Middle East coordinator, and the State Department’s energy envoy, Amos Hochstein, to Riyadh on Wednesday to talk about a range of issues — chief among them the ongoing war in Yemen and global energy supplies. Asked by reporters in Washington whether the U.S.

# Volume timeline

In [ ]:
timeline.plot.line(x='datetime', y='Article Count')